In [5]:
import LinearAlgebra as la

import Random
rng = Random.MersenneTwister();

import ForwardDiff
import ReverseDiff

In [21]:
positions = rand(rng, 5)

5-element Vector{Float64}:
 0.8973642895106613
 0.10633032978951373
 0.43093736976027763
 0.7642183952862915
 0.6446909943794052

In [29]:
function model(positions)
    return sum(positions.^5) + 2
end

model (generic function with 1 method)

## Hessian

In [30]:
const CACHE = Dict{DataType, Any}()

Dict{DataType, Any}()

In [31]:
CACHE

Dict{DataType, Any}()

In [60]:
function inner(y, positions::Array{T}) where {T<:Real}
    if !haskey(CACHE, T)
        config = ReverseDiff.GradientConfig(positions)
        tape = ReverseDiff.compile(ReverseDiff.GradientTape(model, positions, config))
        CACHE[T] = tape
    end
    tape = CACHE[T]
    return ReverseDiff.gradient!(y, tape, positions)
end

function kineticMX(positions, inner, config, result, y)
    ForwardDiff.jacobian!(result, inner, y, positions, config)
    return result #/ SimpleG(positions)
end

kineticMX (generic function with 1 method)

In [61]:
y = similar(x_train)
config = ForwardDiff.JacobianConfig(inner, y, x_train);
result = zeros(5, 5);

In [64]:
@time kineticMX(x_train, inner, config, result, y)

  0.000014 seconds (1 allocation: 224 bytes)


5×5 Matrix{Float64}:
 7.5813  0.0      0.0        0.0   0.0
 0.0     1.51248  0.0        0.0   0.0
 0.0     0.0      0.548285   0.0   0.0
 0.0     0.0      0.0       15.05  0.0
 0.0     0.0      0.0        0.0   5.32321

## Flux model

In [ ]:
model_flux = Flux.Chain(Flux.Dense(5,5),
               Flux.Dense(5,3),
               Flux.Dense(3,1),
               x -> exp.(x)[1])

In [ ]:
@time model_flux(positions)

## Gradient of flux model

In [ ]:
ps = Flux.params(model_flux);

In [ ]:
function grad_flux(model, positions)
    gs = Flux.gradient(() -> model(positions), ps)
    return gs
end

In [ ]:
@time grad_flux_res = grad_flux(model_flux, positions)

In [ ]:
grad_flux_res[model_flux[1].W]